In [31]:
# imports

import os
import io
import sys
import json
import requests
import shutil
import re
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai as genai
import anthropic
from IPython.display import Markdown, display, update_display
import gradio as gr
import subprocess

In [32]:
# environment

load_dotenv(override=True)
os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN', 'your-key-if-not-using-env')
genai_api_key = os.getenv("GOOGLE_API_KEY", "your-gemini-key-if-missing")
genai.configure(api_key=genai_api_key)

In [33]:
# initialize
GEMINI_MODEL = genai.GenerativeModel(model_name="gemini-2.0-flash")

In [34]:
# Define the dropdown options
cpp = gr.Dropdown(['C', 'C++', 'C#', 'Java'], label="Select Programming Language")

In [35]:
def user_prompt_for(python, value):
    user_prompt = f"Rewrite this Python code in {value} with the fastest possible implementation that produces identical output in the least time. "
    user_prompt += f"Respond only with {value} code; do not explain your work other than a few comments. "
    user_prompt += f"Pay attention to number types to ensure no int overflows. Remember to include all necessary {value} packages.\n\n"
    user_prompt += python
    return user_prompt

In [36]:
def messages_for(python, value):
    system_message = f"You are an assistant that reimplements Python code in high performance {value} for an Windows. "
    system_message += f"Respond only with {value} code; use comments sparingly and do not provide any explanation or note or the name of the language in which the code is, other than occasional comments. "
    system_message += f"The {value} response needs to produce an identical output in the fastest possible time. Keep implementations of random number generators identical so that results match exactly."
    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt_for(python, value)}
    ]

In [37]:
# write to a file called optimized.cpp/ optimized.c/ optimized.cs/ optimized.java
def write_output(output, value):
    code = output.replace(f"```{value.lower()}", "").replace("```", "")
    if value == 'C':
        with open("optimized.c", "w") as f:
            f.write(code)
    elif value == 'C++':
        with open("optimized.cpp", "w") as f:
            f.write(code)
    elif value == 'C#':
        with open("optimized.cs", "w") as f:
            f.write(code)
    elif value == 'Java':
        with open("optimized.java", "w") as f:
            f.write(code)

In [38]:
def optimize_gemini(python, value):
    prompt = user_prompt_for(python, value)
    
    # Start streaming response
    response = GEMINI_MODEL.generate_content(
        prompt,
        generation_config={"max_output_tokens": 2000},
        stream=True
    )
    
    reply = ""
    for chunk in response:
        if chunk.text:
            print(chunk.text, end="", flush=True)
            reply += chunk.text

    write_output(reply, value)
    return reply

In [39]:
#Enter your python code here
pi = """
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(100_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
"""

In [40]:
exec(pi)

Result: 3.141592658589
Execution Time: 15.920969 seconds


In [41]:
optimize_gemini(pi, "C")

```c
#include <stdio.h>
#include <time.h>

int main() {
    long long iterations = 100000000LL; // Define iterations as long long
    double param1 = 4.0;
    double param2 = 1.0;
    double result = 1.0;
    clock_t start_time = clock();

    for (long long i = 1; i <= iterations; ++i) {
        double j1 = (double)i * param1 - param2;
        result -= (1.0 / j1);
        double j2 = (double)i * param1 + param2;
        result += (1.0 / j2);
    }

    result *= 4.0;
    clock_t end_time = clock();
    double execution_time = (double)(end_time - start_time) / CLOCKS_PER_SEC;

    printf("Result: %.12f\n", result);
    printf("Execution Time: %.6f seconds\n", execution_time);

    return 0;
}
```

'```c\n#include <stdio.h>\n#include <time.h>\n\nint main() {\n    long long iterations = 100000000LL; // Define iterations as long long\n    double param1 = 4.0;\n    double param2 = 1.0;\n    double result = 1.0;\n    clock_t start_time = clock();\n\n    for (long long i = 1; i <= iterations; ++i) {\n        double j1 = (double)i * param1 - param2;\n        result -= (1.0 / j1);\n        double j2 = (double)i * param1 + param2;\n        result += (1.0 / j2);\n    }\n\n    result *= 4.0;\n    clock_t end_time = clock();\n    double execution_time = (double)(end_time - start_time) / CLOCKS_PER_SEC;\n\n    printf("Result: %.12f\\n", result);\n    printf("Execution Time: %.6f seconds\\n", execution_time);\n\n    return 0;\n}\n```'

In [42]:
def stream_gemini(python_code, value):
    prompt = user_prompt_for(python_code, value)
    system_message = f"You are an assistant that reimplements Python code in high performance {value} for an Windows. "
    system_message += f"Respond only with {value} code; use comments sparingly and do not provide any explanation or note or the name of the language in which the code is, other than occasional comments. "
    system_message += f"The {value} response needs to produce an identical output in the fastest possible time. Keep implementations of random number generators identical so that results match exactly."
    
    response = GEMINI_MODEL.generate_content(
        prompt,
        generation_config={"max_output_tokens": 2000},
        stream=True
    )

    reply = ""
    for chunk in response:
        if chunk.text:
            reply += chunk.text
            yield reply.replace(f"```{value.lower()}\n", '').replace('```', '')

In [43]:
def optimize(python, model, value):
    if model == "gemini-2.0-flash":
        result = stream_gemini(python, value)
    else:
        raise ValueError("Unknown model")
    for stream_so_far in result:
        yield stream_so_far

In [44]:
def execute_python(code):
    try:
        output = io.StringIO()
        sys.stdout = output
        exec(code)
    finally:
        sys.stdout = sys.__stdout__
    return output.getvalue()

In [45]:
def execute_code(code, language):
    """
    Execute code in different programming languages
    """
    try:
        if language == "C":
            return execute_c(code)
        elif language == "C++":
            return execute_cpp(code)
        elif language == "C#":
            return execute_csharp(code)
        elif language == "Java":
            return execute_java(code)
        else:
            return f"Unsupported language: {language}"
    except Exception as e:
        return f"Error executing {language} code: {str(e)}"

In [46]:
def execute_c(code):
    """Execute C code"""
    write_output(code, "C")
    
    # Compile C code
    compiler_cmd = ["gcc", "-O2", "-std=c11", "optimized.c", "-o", "optimized_c"]
    try:
        compile_result = subprocess.run(compiler_cmd, check=True, text=True, capture_output=True)
        
        # Run the compiled program
        run_cmd = ["./optimized_c"] if os.name != 'nt' else ["optimized_c.exe"]
        run_result = subprocess.run(run_cmd, check=True, text=True, capture_output=True)
        
        # Clean up
        cleanup_files(["optimized.c", "optimized_c", "optimized_c.exe"])
        
        return run_result.stdout
    except subprocess.CalledProcessError as e:
        cleanup_files(["optimized.c", "optimized_c", "optimized_c.exe"])
        return f"C compilation/execution error:\n{e.stderr}"

In [47]:
def execute_cpp(code):
    """Execute C++ code"""
    write_output(code, "C++")
    
    # Compile C++ code
    compiler_cmd = ["g++", "-O2", "-std=c++17", "optimized.cpp", "-o", "optimized_cpp"]
    try:
        compile_result = subprocess.run(compiler_cmd, check=True, text=True, capture_output=True)
        
        # Run the compiled program
        run_cmd = ["./optimized_cpp"] if os.name != 'nt' else ["optimized_cpp.exe"]
        run_result = subprocess.run(run_cmd, check=True, text=True, capture_output=True)
        
        # Clean up
        cleanup_files(["optimized.cpp", "optimized_cpp", "optimized_cpp.exe"])
        
        return run_result.stdout
    except subprocess.CalledProcessError as e:
        cleanup_files(["optimized.cpp", "optimized_cpp", "optimized_cpp.exe"])
        return f"C++ compilation/execution error:\n{e.stderr}"

In [48]:
def execute_csharp(code):
    """Execute C# code - simplified version"""
    write_output(code, "C#")
    
    try:
        # Try Mono compiler first (simpler setup)
        if shutil.which("mcs") and shutil.which("mono"):
            # Compile with Mono
            compile_cmd = ["mcs", "optimized.cs"]
            subprocess.run(compile_cmd, check=True, text=True, capture_output=True)
            
            # Run with Mono
            run_cmd = ["mono", "optimized.exe"]
            run_result = subprocess.run(run_cmd, check=True, text=True, capture_output=True)
            
            # Clean up
            cleanup_files(["optimized.cs", "optimized.exe"])
            return run_result.stdout
            
        # Try .NET if Mono is not available
        elif shutil.which("dotnet"):
            # Simple dotnet script execution
            run_cmd = ["dotnet", "script", "optimized.cs"]
            run_result = subprocess.run(run_cmd, check=True, text=True, capture_output=True)
            
            # Clean up
            cleanup_files(["optimized.cs"])
            return run_result.stdout
            
        else:
            return "Error: No C# compiler found. Please install .NET SDK or Mono."
            
    except subprocess.CalledProcessError as e:
        cleanup_files(["optimized.cs", "optimized.exe"])
        return f"C# compilation/execution error:\n{e.stderr}"

In [49]:
def execute_java(code):
    """Execute Java code"""
    write_output(code, "Java")
    
    try:
        # Extract class name from the code
        class_name = extract_java_class_name(code)
        if not class_name:
            return "Error: Could not find public class name in Java code"
        
        # Rename file to match class name
        java_filename = f"{class_name}.java"
        if os.path.exists("optimized.java"):
            os.rename("optimized.java", java_filename)
        
        # Compile Java code
        compile_cmd = ["javac", java_filename]
        subprocess.run(compile_cmd, check=True, text=True, capture_output=True)
        
        # Run Java program
        run_cmd = ["java", class_name]
        run_result = subprocess.run(run_cmd, check=True, text=True, capture_output=True)
        
        # Clean up
        cleanup_files([java_filename, f"{class_name}.class"])
        
        return run_result.stdout
        
    except subprocess.CalledProcessError as e:
        cleanup_files(["optimized.java", f"{class_name}.java", f"{class_name}.class"])
        return f"Java compilation/execution error:\n{e.stderr}"

In [50]:
def extract_java_class_name(code):
    """Extract the public class name from Java code"""
    import re
    match = re.search(r'public\s+class\s+(\w+)', code)
    if match:
        return match.group(1)
    return None

In [51]:
def cleanup_files(filenames):
    """Clean up temporary files"""
    for filename in filenames:
        try:
            if os.path.exists(filename):
                os.remove(filename)
        except:
            pass

In [52]:
css = """
.python {background-color: #306998;}
.cpp {background-color: #050;}
"""

In [53]:
# Gradio interface
with gr.Blocks(css=css) as ui:
    gr.Markdown("<h1 style='text-align: center;'>🔁 Py2Lang</h1><p style='text-align: center;'>Convert code from Python to other languages</p>")
    
    with gr.Row():
        python = gr.Textbox(label="Python code:", value=pi, lines=10)
        converted_code = gr.Textbox(label="Converted code:", lines=10)
    with gr.Row():
        model = gr.Dropdown(["gemini-2.0-flash"], label="Select model", value="gemini-2.0-flash")
        language = gr.Dropdown(['C', 'C++', 'C#', 'Java'], label="Select target language", value="C")
    with gr.Row():
        convert = gr.Button("Convert code")
    with gr.Row():
        python_run = gr.Button("Run Python")
        converted_run = gr.Button("Run Converted Code")
    with gr.Row():
        python_out = gr.TextArea(label="Python result:", elem_classes=["python"])
        converted_out = gr.TextArea(label="Converted code result:")

    convert.click(optimize, inputs=[python, model, language], outputs=[converted_code])
    python_run.click(execute_python, inputs=[python], outputs=[python_out])
    converted_run.click(lambda code, lang: execute_code(code, lang), inputs=[converted_code, language], outputs=[converted_out])

In [54]:
# Launch the interface
if __name__ == "__main__":
    ui.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.
